## COPERNICUS Vegetation Indicatorss 

### <a name="service">Service Definition

In [ ]:
service = dict([('title', 'COPERNICUS Vegetation Indicators - Aggregations'),
                ('abstract', 'COPERNICUS Vegetation Indicators - Aggregations'),
                ('id', 'wfp-01-03-01')])

### <a name="parameter">Parameter Definition 

In [ ]:
N_1 = dict([('id', 'N_1'),
                          ('value', 'True'),
                          ('title', 'No Aggregation'),
                          ('abstract', 'No aggregation')])

In [ ]:
N_3 = dict([('id', 'N_3'),
                          ('value', 'True'),
                          ('title', '30 Day Aggregation'),
                          ('abstract', 'Get a 30 day aggregation')])

In [ ]:
N_6 = dict([('id', 'N_6'),
                          ('value', 'False'),
                          ('title', '60 Day Aggregation'),
                          ('abstract', 'Get a 60 day aggregation')])

In [ ]:
N_9 = dict([('id', 'N_9'),
                          ('value', 'False'),
                          ('title', '90 Day Aggregation'),
                          ('abstract', 'Get a 90 day aggregation')])

In [ ]:
N_12 = dict([('id', 'N_12'),
                          ('value', 'False'),
                          ('title', '120 Day Aggregation'),
                          ('abstract', 'Get a 120 day aggregation')])

In [ ]:
N_15 = dict([('id', 'N_15'),
                          ('value', 'False'),
                          ('title', '150 Day Aggregation'),
                          ('abstract', 'Get a 150 day aggregation')])

In [ ]:
N_18 = dict([('id', 'N_18'),
                          ('value', 'False'),
                          ('title', '180 Day Aggregation'),
                          ('abstract', 'Get a 180 day aggregation')])

In [ ]:
N_27 = dict([('id', 'N_27'),
                          ('value', 'False'),
                          ('title', '270 Day Aggregation'),
                          ('abstract', 'Get a 270 day aggregation')])

In [ ]:
N_36 = dict([('id', 'N_36'),
                          ('value', 'False'),
                          ('title', '360 Day Aggregation'),
                          ('abstract', 'Get a 360 day aggregation')])

In [ ]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                          ('value', 'POLYGON ((26.832 9.5136, 28.6843 9.5136, 28.6843 7.8009, 26.832 7.8009, 26.832 9.5136))'),
                          ('title', 'WKT Polygon for the Region of Interest'),
                          ('abstract', 'Set the value of WKT Polygon')])

In [ ]:
nameOfRegion = dict([('id', 'nameOfRegion'),
                     ('value', ''),
                     ('title', 'Name of Region'),
                     ('abstract', 'Name of the region of interest'),
                     ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input references**

In [13]:
input_references = ('https://catalog.terradue.com/cgls/search?format=atom&uid=fapar_v2_1km_FAPAR-RT6_201702280000_GLOBE_PROBAV_V2.0.2')

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [14]:
import os
import sys
#sys.path.append('/opt/OTB-6.2.0/lib/python')
#sys.path.append('/opt/OTB-6.2.0/lib/libfftw3.so.3')
#os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
#os.environ['LD_LIBRARY_PATH'] = '/opt/OTB-6.2.0/lib'
#os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
#os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'
#import otbApplication
from osgeo import gdal
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame
import math
import re
import cioppy
import shutil
sys.path.append('/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec')
sys.path.append('/application/notebook/libexec')
from aux_functions import calc_average, calc_max_matrix, matrix_sum, crop_image, write_output_image, get_matrix_list

In [15]:
def get_info(row, search_params):
    search = ciop.search(end_point=row['catalogue_url'], 
                                  params=search_params,
                                  output_fields='self,identifier,startdate,enclosure',
                                  model='GeoTime')[0]
    
    series = pd.Series(search)
    
    series['startdate'] = pd.to_datetime(series['startdate'])
    
    return series

In [16]:
#input references used for test only. Remove before deploying to sandbox/production.
'''
input_references = ['https://catalog.terradue.com/cgls/search?format=atom&uid=fapar_v2_1km_FAPAR-RT6_201702280000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=fapar_v2_1km_FAPAR-RT6_201702200000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=fapar_v2_1km_FAPAR-RT6_201702100000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=fapar_v2_1km_FAPAR-RT6_201701310000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=fapar_v2_1km_FAPAR-RT6_201701200000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=fapar_v2_1km_FAPAR-RT6_201701100000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=lai_v2_1km_LAI-RT6_201701310000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=lai_v2_1km_LAI-RT6_201701200000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=lai_v2_1km_LAI-RT6_201701100000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=lai_v2_1km_LAI-RT6_201702280000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=lai_v2_1km_LAI-RT6_201702200000_GLOBE_PROBAV_V2.0.2',
                   'https://catalog.terradue.com/cgls/search?format=atom&uid=lai_v2_1km_LAI-RT6_201702100000_GLOBE_PROBAV_V2.0.2']
'''
if isinstance(input_references, str):
    input_references = [input_references]


region_of_interest = regionOfInterest['value']
nameOfRegion = nameOfRegion['value']
#nameOfRegion = 'Africa'
ciop = cioppy.Cioppy()

lai_references = [ref for ref in input_references if 'LAI' in ref]
fapar_references = [ref for ref in input_references if 'FAPAR' in ref]
gpd_lai_final = None
gpd_fapar_final = None
if lai_references:
    gpd_data = GeoDataFrame(lai_references,
                           columns=['catalogue_url'])

    gpd_data = gpd_data.sort_values(by='catalogue_url')
    print(gpd_data['catalogue_url'])
    start_date = re.findall('\d{8}0000', gpd_data.iloc[0]['catalogue_url'])[0].replace('0000', '')
    end_date = re.findall('\d{8}0000', gpd_data.iloc[-1]['catalogue_url'])[0].replace('0000', '')
    start_date = re.sub(r'(\d{4})(\d{2})(\d{2})', r'\1-\2-\3', start_date)
    end_date = re.sub(r'(\d{4})(\d{2})(\d{2})', r'\1-\2-\3', end_date)
    print(start_date)
    print(end_date)
    print(len(lai_references))
    search_params =  dict([('start', start_date),
                          ('stop', end_date),
                          ('count', len(lai_references))])

    gpd_lai_final = gpd_data.apply(lambda row: get_info(row, search_params), axis=1)

if fapar_references:
    gpd_data = GeoDataFrame(fapar_references,
                           columns=['catalogue_url'])

    gpd_data = gpd_data.sort_values(by='catalogue_url')
    print(gpd_data['catalogue_url'])
    start_date = re.findall('\d{8}0000', gpd_data.iloc[0]['catalogue_url'])[0].replace('0000', '')
    end_date = re.findall('\d{8}0000', gpd_data.iloc[-1]['catalogue_url'])[0].replace('0000', '')
    start_date = re.sub(r'(\d{4})(\d{2})(\d{2})', r'\1-\2-\3', start_date)
    end_date = re.sub(r'(\d{4})(\d{2})(\d{2})', r'\1-\2-\3', end_date)
    print(start_date)
    print(end_date)
    print(len(fapar_references))
    search_params =  dict([('start', start_date),
                          ('stop', end_date),
                          ('count', len(fapar_references))])

    gpd_fapar_final = gpd_data.apply(lambda row: get_info(row, search_params), axis=1)

2    https://catalog.terradue.com/cgls/search?forma...
1    https://catalog.terradue.com/cgls/search?forma...
0    https://catalog.terradue.com/cgls/search?forma...
5    https://catalog.terradue.com/cgls/search?forma...
4    https://catalog.terradue.com/cgls/search?forma...
3    https://catalog.terradue.com/cgls/search?forma...
Name: catalogue_url, dtype: object
2017-01-10
2017-02-28
6
5    https://catalog.terradue.com/cgls/search?forma...
4    https://catalog.terradue.com/cgls/search?forma...
3    https://catalog.terradue.com/cgls/search?forma...
2    https://catalog.terradue.com/cgls/search?forma...
1    https://catalog.terradue.com/cgls/search?forma...
0    https://catalog.terradue.com/cgls/search?forma...
Name: catalogue_url, dtype: object
2017-01-10
2017-02-28
6


In [17]:
'''
test_lta = np.array([['', 'start_date', 'end_date', 'enclosure'],
                    ['1', '2015.01.10', '2017.01.31', 'LTA.LAI.Sudan.N3.max_values.2015.1.10.2017.1.31.tif'],
                    ['2', '2015.01.10', '2017.01.31', 'LTA.FAPAR.Sudan.N3.averages.2015.1.10.2017.1.31.tif']])

lta_data = GeoDataFrame(data=test_lta[1:,1:],
                        index=test_lta[1:,0],
                        columns=test_lta[0,1:])
lta_data['start_date'] = pd.to_datetime(lta_data['start_date'])
lta_data['end_date'] = pd.to_datetime(lta_data['end_date'])
print(lta_data)
'''

"\ntest_lta = np.array([['', 'start_date', 'end_date', 'enclosure'],\n                    ['1', '2015.01.10', '2017.01.31', 'LTA.LAI.Sudan.N3.max_values.2015.1.10.2017.1.31.tif'],\n                    ['2', '2015.01.10', '2017.01.31', 'LTA.FAPAR.Sudan.N3.averages.2015.1.10.2017.1.31.tif']])\n\nlta_data = GeoDataFrame(data=test_lta[1:,1:],\n                        index=test_lta[1:,0],\n                        columns=test_lta[0,1:])\nlta_data['start_date'] = pd.to_datetime(lta_data['start_date'])\nlta_data['end_date'] = pd.to_datetime(lta_data['end_date'])\nprint(lta_data)\n"

In [18]:
'''
def get_lta_from_dataframe(dataframe, product_type, aggregation, name_of_region, start_day, end_day, start_month, end_month):
    dataframe_agr = dataframe[(dataframe['enclosure'].str.contains(aggregation)) &
                              (dataframe['enclosure'].str.contains(product_type)) &
                              (dataframe['enclosure'].str.contains(name_of_region))]
    return dataframe_agr[(dataframe_agr['start_date'].dt.day == start_day) & 
                     (dataframe_agr['start_date'].dt.month == start_month) & 
                     (dataframe_agr['end_date'].dt.day == end_day) & 
                     (dataframe_agr['end_date'].dt.month == end_month)]['enclosure'].tolist()
'''

"\ndef get_lta_from_dataframe(dataframe, product_type, aggregation, name_of_region, start_day, end_day, start_month, end_month):\n    dataframe_agr = dataframe[(dataframe['enclosure'].str.contains(aggregation)) &\n                              (dataframe['enclosure'].str.contains(product_type)) &\n                              (dataframe['enclosure'].str.contains(name_of_region))]\n    return dataframe_agr[(dataframe_agr['start_date'].dt.day == start_day) & \n                     (dataframe_agr['start_date'].dt.month == start_month) & \n                     (dataframe_agr['end_date'].dt.day == end_day) & \n                     (dataframe_agr['end_date'].dt.month == end_month)]['enclosure'].tolist()\n"

In [19]:
def calc_aggregations(product_list, N_value, region_of_interest, product_type):
    mask_no_data_value = 0
    max_values = 0
    averages = 0
    temp_list = []
    no_data_value = None
    print(type(product_list))
    for product_url in product_list:
        # uncompressed data
        product = (product_url.split('/')[-1]).split('.gz')[0]
        print(product)
        cropped_product_path = 'crop_' + product
        cropped_product_path = cropped_product_path.split('.nc')[0] + '.tif'
        try:
            crop_image(product_url, region_of_interest, cropped_product_path, product_type)
            # Read GeoTIFF as an array
            dataset = gdal.Open(cropped_product_path)
            product_array = dataset.GetRasterBand(1).ReadAsArray()
            no_data_value = dataset.GetRasterBand(1).GetNoDataValue()
            print(no_data_value)
            geo_transform = dataset.GetGeoTransform()
            projection = dataset.GetProjection()
            ## Create mask of no_data_values
            if no_data_value is not None:
                if isinstance(mask_no_data_value, int):
                    mask_no_data_value = np.where(product_array == no_data_value, 1, 0)
                else:
                    temp_mask = np.where(product_array == no_data_value, 1, 0)
                    mask_no_data_value = matrix_sum(mask_no_data_value, temp_mask)
            max_values = calc_max_matrix(max_values, product_array, no_data_value)
            temp_list.append(product_array)
            dataset = None
            
        except Exception as e:
            print('Error processing the product ' + product + ': ' + str(e))
        if os.path.exists(cropped_product_path):
            os.remove(cropped_product_path)
    
    averages = calc_average(temp_list, N_value, no_data_value)
    
    return max_values, averages, mask_no_data_value, geo_transform, projection

In [20]:
def product_stage_in(enclosure, target_dir):
    if not os.path.isdir(target_dir):
        os.mkdir(target_dir)
    return ciop.copy(urls=enclosure, target=target_dir)

In [21]:
def write_outputs(product_name, roi_name, first_date, last_date, averages, max_values, mask_no_data_value, image_format, product_count, projection, geo_transform, no_data_value):
    filenames = []
    filenames.append(product_name + '_' + roi_name + '_N' + str(product_count) + '_averages_' + first_date + '_' + last_date + '.tif')
    filenames.append(product_name + '_' + roi_name + '_N' + str(product_count) + '_maxvalues_' + first_date + '_' + last_date + '.tif')
    write_output_image(filenames[0], averages, image_format,  mask_no_data_value, projection, geo_transform, no_data_value)
    write_output_image(filenames[1], max_values, image_format, mask_no_data_value, projection, geo_transform, no_data_value)
    return filenames

In [22]:
def get_formatted_date(product_reference):
    metadata = ciop.search(end_point=product_reference,
                           params=[],
                           output_fields='identifier,startdate',
                           model="GeoTime")
    return metadata[0]['startdate']

In [23]:
def write_properties_file(dataframe, output_name):
    
    title = 'Output %s' % output_name
    first_date = get_formatted_date(dataframe.iloc[0]['self'])
    last_date = get_formatted_date(dataframe.iloc[-1]['self'])
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (regionOfInterest['value']))

In [24]:
nlist = [N_1['value'], N_3['value'], N_6['value'], N_9['value'], N_12['value'], N_15['value'], N_18['value'], N_27['value'], N_36['value']]
nvalue = [1, 3, 6, 9, 12, 15, 18, 27, 36]
#L = len(gpd_final.index.values)
nlist = [n=='True' for n in nlist]
print(nlist)
if gpd_lai_final is not None:
    lai_product_years = gpd_lai_final['startdate'].dt.year.unique()
else:
    lai_product_years = []
if gpd_fapar_final is not None:
    fapar_product_years = gpd_fapar_final['startdate'].dt.year.unique()
else:
    fapar_product_years = []
product_years = list(lai_product_years) + list(set(fapar_product_years) - set(lai_product_years))
print(product_years)
for _year in product_years:
    #products_data = gpd_final[(gpd_final['startdate'].dt.year == _year)]
    #lai_products = products_data[(products_data['identifier'].str.contains('lai'))]
    #fapar_products = products_data[(products_data['identifier'].str.contains('fapar'))]
    lai_products = gpd_lai_final[(gpd_lai_final['startdate'].dt.year == _year)]
    fapar_products = gpd_fapar_final[(gpd_fapar_final['startdate'].dt.year == _year)]
    if not lai_products.empty:
        products_data = lai_products
        L = len(products_data.index.values)
        months_of_products = products_data['startdate'].dt.month.unique()
        print(months_of_products)
        for n in [index for index, value in enumerate(nlist) if value==True]:
            N = nvalue[n]
            n_months = N/3
            if N == 1:
                pass
            else: 
                if N == 3:
                    for month_idx in months_of_products:
                        print(month_idx)
                        start_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate']
                        end_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[-1]['startdate']
                        interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                        interval_products = interval_gpd['enclosure'].tolist()
                        
                        file_list = []
                        for enclosure in interval_products:
                            filepath = product_stage_in(enclosure, 'tmp_data')
                            file_list.append(filepath)
                            print(file_list)
                        if file_list:
                            first_date = start_date.strftime('%Y-%m-%d')
                            last_date = end_date.strftime('%Y-%m-%d')
                            print(first_date)
                            print(last_date)
                            max_values, averages, no_value, geo_transform, projection = calc_aggregations(file_list, N, region_of_interest, 'LAI')
                        
                            '''
                            LTA_max_values = get_lta_from_dataframe(lta_data, 'LAI', 'max_values', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month)
                            LTA_averages = get_lta_from_dataframe(lta_data, 'LAI', 'averages', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month)

                            print('Calculating anomalies...')
                            if LTA_max_values:
                                LTA_max_values = get_matrix_list(LTA_max_values)[0]
                                anomaly_max_values = np.divide(max_values, LTA_max_values)
                                print(anomaly_max_values)
                            if LTA_averages:
                                LTA_averages = get_matrix_list(LTA_averages)[0]
                                anomaly_averages = np.divide(averages, LTA_averages)
                                print(anomaly_averages)
                            '''
                        
                            filenames = write_outputs('LAI', nameOfRegion, first_date, last_date, averages, max_values, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                            print(filenames)
                            for output_name in filenames:
                                write_properties_file(interval_gpd, output_name)
                            for tmp_file in file_list:
                                os.remove(tmp_file)
                        else:
                            print('ERROR: No LAI product files found.')
                else:
                    for month_idx in range(months_of_products[0], months_of_products[-1], n_months):
                        start_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate']
                        end_date = products_data[(products_data['startdate'].dt.month == month_idx + n_months-1)].iloc[-1]['startdate']
                        interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                        interval_products = interval_gpd['enclosure'].tolist()
                        
                        file_list = []
                        for enclosure in interval_products:
                            filepath = product_stage_in(enclosure, 'tmp_data')
                            file_list.append(filepath)
                            print(file_list)
                        if file_list:
                            first_date = start_date.strftime('%Y-%m-%d')
                            last_date = end_date.strftime('%Y-%m-%d')
                            print(start_date)
                            print(end_date)
                            max_values, averages, no_value, geo_transform, projection = calc_aggregations(file_list, N, region_of_interest, 'LAI')
                        
                            '''
                            LTA_max_values = get_lta_from_dataframe(lta_data, 'LAI', 'max_values', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month)
                            LTA_averages = get_lta_from_dataframe(lta_data, 'LAI', 'averages', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month)

                            print('Calculating anomalies...')
                            if LTA_max_values:
                                LTA_max_values = get_matrix_list(LTA_max_values)[0]
                                anomaly_max_values = np.divide(max_values, LTA_max_values)
                                print(anomaly_max_values)
                            if LTA_averages:
                                LTA_averages = get_matrix_list(LTA_averages)[0]
                                anomaly_averages = np.divide(averages, LTA_averages)
                                print(anomaly_averages)
                            '''
                        
                            filenames = write_outputs('LAI', nameOfRegion, first_date, last_date, averages, max_values, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                            for output_name in filenames:
                                write_properties_file(interval_gpd, output_name)
                            for tmp_file in file_list:
                                os.remove(tmp_file)
                        else:
                            print('ERROR: No LAI product files found.')
    
    if not fapar_products.empty:
        products_data = fapar_products
        L = len(products_data.index.values)
        months_of_products = products_data['startdate'].dt.month.unique()
        print(months_of_products)
        for n in [index for index, value in enumerate(nlist) if value==True]:
            N = nvalue[n]
            n_months = N/3
            if N == 1:
                pass
            else: 
                if N == 3:
                    for month_idx in months_of_products:
                        print(month_idx)
                        start_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate']
                        end_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[-1]['startdate']
                        interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                        interval_products = interval_gpd['enclosure'].tolist()
                        
                        file_list = []
                        for enclosure in interval_products:
                            filepath = product_stage_in(enclosure, 'tmp_data')
                            file_list.append(filepath)
                            print(file_list)
                        if file_list:
                            first_date = start_date.strftime('%Y-%m-%d')
                            last_date = end_date.strftime('%Y-%m-%d')
                            print(start_date)
                            print(end_date)
                            max_values, averages, no_value, geo_transform, projection = calc_aggregations(file_list, N, region_of_interest, 'FAPAR')
                        
                            '''
                            LTA_max_values = get_lta_from_dataframe(lta_data, 'LAI', 'max_values', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month)
                            LTA_averages = get_lta_from_dataframe(lta_data, 'LAI', 'averages', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month)

                            print('Calculating anomalies...')
                            if LTA_max_values:
                                LTA_max_values = get_matrix_list(LTA_max_values)[0]
                                anomaly_max_values = np.divide(max_values, LTA_max_values)
                                print(anomaly_max_values)
                            if LTA_averages:
                                LTA_averages = get_matrix_list(LTA_averages)[0]
                                anomaly_averages = np.divide(averages, LTA_averages)
                                print(anomaly_averages)
                            '''
                        
                            filenames = write_outputs('FAPAR', nameOfRegion, first_date, last_date, averages, max_values, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                            for output_name in filenames:
                                write_properties_file(interval_gpd, output_name)
                            for tmp_file in file_list:
                                os.remove(tmp_file)
                        else:
                            print('ERROR: No fAPAR product files found.')
                    
                else:
                    for month_idx in range(months_of_products[0], months_of_products[-1], n_months):
                        print(month_idx)
                        start_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate']
                        end_date = products_data[(products_data['startdate'].dt.month == month_idx + n_months-1)].iloc[-1]['startdate']
                        interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                        interval_products = interval_gpd['enclosure'].tolist()
                        
                        file_list = []
                        for enclosure in interval_products:
                            filepath = product_stage_in(enclosure, 'tmp_data')
                            file_list.append(filepath)
                            print(file_list)
                        if file_list:
                            first_date = start_date.strftime('%Y-%m-%d')
                            last_date = end_date.strftime('%Y-%m-%d')
                            print(start_date)
                            print(end_date)
                            max_values, averages, no_value, geo_transform, projection = calc_aggregations(file_list, N, region_of_interest, 'FAPAR')
                        
                            '''
                            LTA_max_values = get_lta_from_dataframe(lta_data, 'LAI', 'max_values', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month)
                            LTA_averages = get_lta_from_dataframe(lta_data, 'LAI', 'averages', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month)

                            print('Calculating anomalies...')
                            if LTA_max_values:
                                LTA_max_values = get_matrix_list(LTA_max_values)[0]
                                anomaly_max_values = np.divide(max_values, LTA_max_values)
                                print(anomaly_max_values)
                            if LTA_averages:
                                LTA_averages = get_matrix_list(LTA_averages)[0]
                                anomaly_averages = np.divide(averages, LTA_averages)
                                print(anomaly_averages)
                            '''
                        
                        
                            filenames = write_outputs('FAPAR', nameOfRegion, first_date, last_date, averages, max_values, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                            for output_name in filenames:
                                write_properties_file(interval_gpd, output_name)
                            for tmp_file in file_list:
                                os.remove(tmp_file)
                        else:
                            print('ERROR: No fAPAR product files found.')   

[True, True, False, False, False, False, False, False, False]
[2017]
[1 2]
1
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701100000_GLOBE_PROBAV_V2.0.2.nc']
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701100000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701200000_GLOBE_PROBAV_V2.0.2.nc']
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701100000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701200000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701310000_GLOBE_PROBAV_V2.0.2.nc']
2017-01-10
2017-01-31
<type 'list'>
c_gls_LAI-RT6_201701100000_GLOBE_PROBAV_V2.0.2.nc
255.0
c_gls_LAI-RT6_201701200000_GLOBE_PROBAV_V2.0.2.nc
255.0
c_gls_LAI-RT6_20

Exception: subprocess returned: 1
2019-02-06T12:34:00 [INFO   ][ciop-copy][starting] url 'https://store.terradue.com/cgls/fapar_v2_1km/2017/02/10/files/v1/c_gls_FAPAR-RT6_201702100000_GLOBE_PROBAV_V2.0.2.nc' > local '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/'
2019-02-06T12:34:01 [ERROR  ][ciop-copy][failed] sink '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_FAPAR-RT6_201702100000_GLOBE_PROBAV_V2.0.2.nc' already exists


In [ ]:
'''from matplotlib import pyplot
%matplotlib inline


fig0 = pyplot.figure()
ax0 = fig0.add_subplot(111)
cax0 = ax0.matshow(max_values)
fig0.colorbar(cax0)

fig1 = pyplot.figure()
ax1 = fig1.add_subplot(111)
cax1 = ax1.matshow(averages)
fig1.colorbar(cax1)


fig = pyplot.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(no_value)
fig.colorbar(cax)

pyplot.show()
'''

### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.